In [3]:
import numpy as np
import pandas as pd
from pandas_datareader import data

In [4]:
!pip install pandas_datareader

In [5]:
import pandas as pd
company="MSFT"
start='2022-05-15'
end='2022-10-15'

stocks=data.DataReader(name=company, data_source='yahoo', start=start, end=end)
pd.PeriodIndex(stocks.index, freq='D')

PeriodIndex(['2022-05-16', '2022-05-17', '2022-05-18', '2022-05-19',
             '2022-05-20', '2022-05-23', '2022-05-24', '2022-05-25',
             '2022-05-26', '2022-05-27',
             ...
             '2022-10-03', '2022-10-04', '2022-10-05', '2022-10-06',
             '2022-10-07', '2022-10-10', '2022-10-11', '2022-10-12',
             '2022-10-13', '2022-10-14'],
            dtype='period[D]', name='Date', length=106, freq='D')

In [6]:
stocks.truncate(before='22-05-15', after='2022-10-15' )
stocks.reset_index(inplace = True)
stocks.head(5)

,Date,High,Low,Open,Close,Volume,Adj Close
0,2022-05-16,265.820007,255.779999,259.959991,261.500000,32550900.0,260.339752
1,2022-05-17,268.329987,262.459991,266.109985,266.820007,28828800.0,265.636169
2,2022-05-18,263.600006,252.770004,263.000000,254.080002,31356000.0,253.541840
3,2022-05-19,257.670013,251.880005,253.899994,253.139999,32692300.0,252.603821
4,2022-05-20,258.540009,246.440002,257.239990,252.559998,39199300.0,252.025055


In [7]:
len(stocks.Date)

106